In [41]:
import requests
import time
import pymysql
import os
import json

from rdflib import Graph, ConjunctiveGraph, Literal, BNode, Namespace, RDF, URIRef,RDFS
from rdflib.namespace import DC, FOAF, XSD
from owlrl import DeductiveClosure, RDFS_Semantics
import pprint

In [2]:
conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')
cursor=conn.cursor()

### Getting list of repositories and authors from the file

In [3]:
cursor.execute("SHOW COLUMNS FROM krw.paper_author")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    print(r)

('paper_url', 'varchar(255)', 'NO', 'MUL', None, '')
('author', 'varchar(255)', 'NO', 'MUL', None, '')


In [4]:
data = []
paper_ID = {}
cursor.execute("SELECT krw.repo.paper_url, repo_url, author, paper_title, irepoID \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 10000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))
    
author_dict = {}
git_dict = {}
# Version with name as one string
for d in data:
    if d[1] not in author_dict:
        author_dict[d[1]]=[d[2]]
        git_dict[d[1]]=d[3]
        paper_ID[d[1]]=d[4] 
    else:
        author_dict[d[1]].append(d[2])
print(len(author_dict.keys()))

2079


In [5]:
paper_ID

{'https://github.com/nianticlabs/monodepth2': 157101,
 'https://github.com/google/tf_mesh_renderer': 157102,
 'https://github.com/Panda-Peter/visda2019-multisource': 157103,
 'https://github.com/yanggeng1995/GAN-TTS': 157104,
 'https://github.com/twke18/Adaptive_Affinity_Fields': 157105,
 'https://github.com/adelra/query-expansion': 157106,
 'https://github.com/JunMa11/MICCAI2019-OpenSourcePapers': 157107,
 'https://github.com/KlugerLab/t-SNE-Heatmaps': 157108,
 'https://github.com/VladyslavUsenko/basalt-mirror': 157109,
 'https://github.com/arthurxlw/cytonRL': 157110,
 'https://github.com/vwz/topolstm': 157114,
 'https://github.com/tianzhi0549/FCOS': 157115,
 'https://github.com/new-okaerinasai/bwgan_pytorch': 157116,
 'https://github.com/ManuelVs/NeuralNetworks': 157117,
 'https://github.com/SJTU-lqiu/QA4IE': 157118,
 'https://github.com/ctuning/reproduce-ck-paper': 157119,
 'https://github.com/ruotianluo/DiscCaptioning': 157120,
 'https://github.com/psychopa4/PFNL': 157121,
 'https:

In [45]:
filename = 'scraped_data.txt' # use full version later

In [46]:
paper_data = {}

with open(filename) as fp:
    line = fp.readline().split()
    current_repo = line[0]
    contributors = [line[1:]]
    while line:
        line = fp.readline().split()
        if len(line) and line[0]==current_repo:
            contributors.append(line[1:])
        else:
            print(f'Doing {current_repo} with {len(contributors)} contributors')
            print(f'Authors: {author_dict[current_repo]}')
            paper_data[git_dict[current_repo]] = {}
            for cont in contributors:
                author_name = cont[1].replace("_"," ")
                found_name = author_name in author_dict[current_repo]
                print(f"{author_name} in {found_name}")
                if found_name:
                    paper_data[git_dict[current_repo]][author_name] = cont[2:]
            # after saving previous authors, load new repo
            if len(line):
                current_repo = line[0]
                contributors = [line[1:]]         

Doing https://github.com/google/tf_mesh_renderer with 3 contributors
Authors: ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
NULL in False
Kyle Genova in True
NULL in False
Doing https://github.com/twke18/Adaptive_Affinity_Fields with 3 contributors
Authors: ['Tsung-Wei Ke', 'Jyh-Jing Hwang', 'Ziwei Liu', 'Stella X. Yu']
Tsung-Wei Ke in True
Jyh-Jing Hwang in True
Tsung-Wei Ke in True
Doing https://github.com/VladyslavUsenko/basalt-mirror with 3 contributors
Authors: ['Vladyslav Usenko', 'Nikolaus Demmel', 'Daniel Cremers', 'David Schubert', 'Thore Goll', 'Nikolaus Demmel', 'Vladyslav Usenko', 'Jörg Stückler', 'Daniel Cremers', 'Vladyslav Usenko', 'Nikolaus Demmel', 'David Schubert', 'Jörg Stückler', 'Daniel Cremers', 'Christiane Sommer', 'Vladyslav Usenko', 'David Schubert', 'Nikolaus Demmel', 'Daniel Cremers']
Vladyslav Usenko in True
Nikolaus Demmel in True
NULL in False
Doing https://github.com/new-okaerinasai/bwgan_pytorc

KeyError: 'https://github.com/benedekrozemberczki/karateclub'

In [47]:
paper_data

{'Unsupervised Training for 3D Morphable Model Regression': {'Kyle Genova': ['2',
   '18',
   '1']},
 'Adaptive Affinity Fields for Semantic Segmentation': {'Tsung-Wei Ke': ['2',
   '24',
   '6'],
  'Jyh-Jing Hwang': ['9', '31', '26']},
 'The Double Sphere Camera Model': {'Vladyslav Usenko': ['227',
   '66,951',
   '10,162'],
  'Nikolaus Demmel': ['24', '662', '1,219']},
 'Banach Wasserstein GAN': {},
 'Convolutional Neural Networks for Sentence Classification': {},
 'Bottom-Up and Top-Down Attention for Image Captioning and Visual Question Answering': {},
 'FeaStNet: Feature-Steered Graph Convolutions for 3D Shape Analysis': {'Nitika Verma': ['5',
   '344',
   '11']},
 'Deep Reinforcement Learning for Dialogue Generation': {},
 'Unsupervised Feature Selection with Adaptive Structure Learning': {},
 'SGDR: Stochastic Gradient Descent with Warm Restarts': {'Ilya Loshchilov': ['4',
   '461',
   '2']},
 'SSD: Single Shot MultiBox Detector': {},
 'An attentive neural architecture for joint

### Loading IDs from JSON

In [42]:
ns = Namespace("http://www.scoaring.com/ontology#")
owl = Namespace("http://www.w3.org/2002/07/owl#")

In [17]:
# change encoding of file
'''
f = open("datav4.ttl", "r", encoding='latin-1')
output = f.read()
f = open("data.ttl", "a")
f.write(output)
f.close()
'''

In [19]:
g = Graph()
g.parse("data.ttl", format="ttl")

<Graph identifier=N26ef879268cc4635b82f98dd65d30dbf (<class 'rdflib.graph.Graph'>)>

In [12]:
:A145415	rdf:type	owl:NamedIndividual :author;

{'Masked Autoregressive Flow for Density Estimation': rdflib.term.URIRef('http://www.scoaring.com/ontology#P1241734'),
 'Bayesian optimisation for fast approximate inference in state-space models with intractable likelihoods': rdflib.term.URIRef('http://www.scoaring.com/ontology#P1278149'),
 'iCaRL: Incremental Classifier and Representation Learning': rdflib.term.URIRef('http://www.scoaring.com/ontology#P1239024'),
 'On the Quantitative Analysis of Decoder-Based Generative Models': rdflib.term.URIRef('http://www.scoaring.com/ontology#P1184876'),
 'Deep Layer Aggregation': rdflib.term.URIRef('http://www.scoaring.com/ontology#P1243354'),
 'Building Generalizable Agents with a Realistic and Rich 3D Environment': rdflib.term.URIRef('http://www.scoaring.com/ontology#P1271715'),
 'Learning to Discover Cross-Domain Relations with Generative Adversarial Networks': rdflib.term.URIRef('http://www.scoaring.com/ontology#P1226649'),
 'High-Order Attention Models for Visual Question Answering': rdfl

In [21]:
paper_id = {}

for s, p, o in g.triples((None, dc.title, None)):
    print(s, o)
    paper_id[str(o)]=s

http://www.scoaring.com/ontology#P1232978 Using millions of emoji occurrences to learn any-domain representations for detecting sentiment, emotion and sarcasm
http://www.scoaring.com/ontology#P1242196 Porting an Open Information Extraction System from English to German
http://www.scoaring.com/ontology#P1252701 Focal Loss for Dense Object Detection
http://www.scoaring.com/ontology#P1184372 Spatio-temporal video autoencoder with differentiable memory
http://www.scoaring.com/ontology#P1186411 Action Recognition using Visual Attention
http://www.scoaring.com/ontology#P1282292 Corpus Selection Approaches for Multilingual Parsing from Raw Text to Universal Dependencies
http://www.scoaring.com/ontology#P1278548 Efficiently Trainable Text-to-Speech System Based on Deep Convolutional Networks with Guided Attention
http://www.scoaring.com/ontology#P1278471 CTAP: Complementary Temporal Action Proposal Generation
http://www.scoaring.com/ontology#P1192890 GXNOR-Net: Training deep neural networks wi

In [26]:
authors = {}
for s, p, o in g.triples((None, RDF.type, ns.author)):
    for s1, p1, o1 in g.triples((s, FOAF.name, None)):
        print(s, o1)
        authors[str(o1)]=s

http://www.scoaring.com/ontology#A16964
http://www.scoaring.com/ontology#A16964 Basir Shariat
http://www.scoaring.com/ontology#A89896
http://www.scoaring.com/ontology#A89896 Mark Gales
http://www.scoaring.com/ontology#A137552
http://www.scoaring.com/ontology#A137552 Taco S Cohen
http://www.scoaring.com/ontology#A42909
http://www.scoaring.com/ontology#A42909 Felipe Llinares-Lopez
http://www.scoaring.com/ontology#A112244
http://www.scoaring.com/ontology#A112244 Pietro Liò
http://www.scoaring.com/ontology#A145142
http://www.scoaring.com/ontology#A145142 Vasileios Mezaris
http://www.scoaring.com/ontology#A109989
http://www.scoaring.com/ontology#A109989 Pedro V. Sander
http://www.scoaring.com/ontology#A26790
http://www.scoaring.com/ontology#A26790 Chu-Song Chen
http://www.scoaring.com/ontology#A121336
http://www.scoaring.com/ontology#A121336 Rui Yan
http://www.scoaring.com/ontology#A07012
http://www.scoaring.com/ontology#A07012 Alina Oprea
http://www.scoaring.com/ontology#A23270
http://www.

http://www.scoaring.com/ontology#A70248
http://www.scoaring.com/ontology#A70248 Jorge Herrera
http://www.scoaring.com/ontology#A87323
http://www.scoaring.com/ontology#A87323 Mani Srivastava
http://www.scoaring.com/ontology#A94360
http://www.scoaring.com/ontology#A94360 Meike Zehlike
http://www.scoaring.com/ontology#A98512
http://www.scoaring.com/ontology#A98512 Miryam de Lhoneux
http://www.scoaring.com/ontology#A60916
http://www.scoaring.com/ontology#A60916 Jakob Wasserthal
http://www.scoaring.com/ontology#A143352
http://www.scoaring.com/ontology#A143352 Tsui-Wei Weng
http://www.scoaring.com/ontology#A121679
http://www.scoaring.com/ontology#A121679 Ruo-Ping Dong
http://www.scoaring.com/ontology#A32347
http://www.scoaring.com/ontology#A32347 David M. Blei
http://www.scoaring.com/ontology#A105066
http://www.scoaring.com/ontology#A105066 Niranjan Balasubramanian
http://www.scoaring.com/ontology#A137822
http://www.scoaring.com/ontology#A137822 Tak-Wai Hui
http://www.scoaring.com/ontology#A

http://www.scoaring.com/ontology#A42376
http://www.scoaring.com/ontology#A42376 Faruk Ahmed
http://www.scoaring.com/ontology#A26576
http://www.scoaring.com/ontology#A26576 Christopher Nemeth
http://www.scoaring.com/ontology#A149433
http://www.scoaring.com/ontology#A149433 Wessel Bruinsma
http://www.scoaring.com/ontology#A05349
http://www.scoaring.com/ontology#A05349 Alex Sandy Pentland
http://www.scoaring.com/ontology#A56916
http://www.scoaring.com/ontology#A56916 Huy Trinh
http://www.scoaring.com/ontology#A103225
http://www.scoaring.com/ontology#A103225 Ngai Wong
http://www.scoaring.com/ontology#A30442
http://www.scoaring.com/ontology#A30442 Daniel Khashabi
http://www.scoaring.com/ontology#A67757
http://www.scoaring.com/ontology#A67757 Jo-Han Hsu
http://www.scoaring.com/ontology#A18954
http://www.scoaring.com/ontology#A18954 Binod Bhattarai
http://www.scoaring.com/ontology#A64336
http://www.scoaring.com/ontology#A64336 Jesse Dunietz
http://www.scoaring.com/ontology#A87935
http://www.s

http://www.scoaring.com/ontology#A105851 Núria Queralt-Rosinach
http://www.scoaring.com/ontology#A23132
http://www.scoaring.com/ontology#A23132 Changshui Zhang
http://www.scoaring.com/ontology#A23799
http://www.scoaring.com/ontology#A23799 Chee Seng Chan
http://www.scoaring.com/ontology#A123265
http://www.scoaring.com/ontology#A123265 Saeed Shiry Ghidary
http://www.scoaring.com/ontology#A145939
http://www.scoaring.com/ontology#A145939 Vijay Kumar
http://www.scoaring.com/ontology#A47776
http://www.scoaring.com/ontology#A47776 Gil Elbaz
http://www.scoaring.com/ontology#A44331
http://www.scoaring.com/ontology#A44331 Francis Bond
http://www.scoaring.com/ontology#A153454
http://www.scoaring.com/ontology#A153454 Xuehan Xiong
http://www.scoaring.com/ontology#A52467
http://www.scoaring.com/ontology#A52467 Haoqian Wang
http://www.scoaring.com/ontology#A70594
http://www.scoaring.com/ontology#A70594 Josef Psutka
http://www.scoaring.com/ontology#A111193
http://www.scoaring.com/ontology#A111193 Pet

### Creating graph

In [50]:
software_graph = Graph()
software_graph.add((ns.softwareContribution, RDFS.subClassOf, ns.contribution))
for paper in paper_data.keys():
    if paper in paper_id:
        print(paper, paper_id[paper])
        for author in paper_data[paper].keys():
            if author in authors:
                print(authors[author].split("#")[-1])
                contr_id = URIRef(f"{paper_id[paper]}_{authors[author].split('#')[-1]}_software")
                print(authors[author], contr_id)
                software_graph.add((contr_id, RDF.type, ns.softwareContribution))
                software_graph.add((contr_id, RDF.type, owl.NamedIndividual))
                software_graph.add((contr_id, ns.isContributionByAuthor, authors[author]))
                software_graph.add((contr_id, ns.isContributionToPaper, paper_id[paper]))
                software_graph.add((contr_id, ns.role, Literal("software")))
                software_graph.add((contr_id, ns.numberOfCommits, Literal(paper_data[paper][author][0], datatype=XSD.integer)))
                software_graph.add((contr_id, ns.numberOfLinesAddedToRepo, Literal(paper_data[paper][author][1], datatype=XSD.integer)))
                software_graph.add((contr_id, ns.numberOfLinesDeleted, Literal(paper_data[paper][author][2], datatype=XSD.integer)))
    else:
        print('Not in the list')

Not in the list
Not in the list
Not in the list
Not in the list
Not in the list
Not in the list
FeaStNet: Feature-Steered Graph Convolutions for 3D Shape Analysis http://www.scoaring.com/ontology#P1237474
A105231
http://www.scoaring.com/ontology#A105231 http://www.scoaring.com/ontology#P1237474_A105231_software
Not in the list
Not in the list
Not in the list
Not in the list
An attentive neural architecture for joint segmentation and parsing and its application to real estate ads http://www.scoaring.com/ontology#P1280813
A47715
http://www.scoaring.com/ontology#A47715 http://www.scoaring.com/ontology#P1280813_A47715_software
Emergent Complexity via Multi-Agent Competition http://www.scoaring.com/ontology#P1228876
A143085
http://www.scoaring.com/ontology#A143085 http://www.scoaring.com/ontology#P1228876_A143085_software
Not in the list
Not in the list
Not in the list
Not in the list
Not in the list
Not in the list
Not in the list
Relational Neural Expectation Maximization: Unsupervised Di

In [52]:
software_graph.serialize(destination='software_contributions.ttl', format='ttl')